Попробуем решить следующую задачу: нам необходимо выгрузить сайт NPlus1.ru и положить статьи в базу данных. Отдельно должны лежать идентификаторы загруженных текстов, словари начальных форм и токенов, разбиение текста на предложения. Для решения задачи будем использовать базу данных MongoDB.

Импортируем все необходисые библиотеки. Обратите особое внимание на библиотеку для работы с Mongo - <a href="http://api.mongodb.com/python/current/api/index.html">pymongo</a> (еще одно описание <a href="https://pymongo.readthedocs.io/en/stable/">здесь</a>).

In [86]:
import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm import tqdm
import pymorphy2

import pymongo
from bson import ObjectId
from lxml import html

delcom=re.compile("<!--.+-->", re.S)


Чтобы не ставить себе базу данных, можно использовать бесплатное облако <a href ="https://cloud.mongodb.com/">Mongo.Atlas</a>. Для этого необходимо зарегистрироваться, завести пользователя, создать проект, дать пользователю права на проект. Для пользователя получить строку для соединения с базой. 

Но в данном случае мы будем пользоваться MongoDB, запущенное на локальном компьютере при помощи Docker.

Для работы с БД используем класс MongoClient, в который передаем строку соединения.

In [87]:
# Version after 3.6
client = pymongo.MongoClient('localhost', 27017)
#client = pymongo.MongoClient()

Аналогом реляционных записей являются документы. Аналогом таблиц - коллекции и подколлекции. Сам документ может включать в себя другие документы. Документ записывается в формате JSON, но для Питона можно считать, что это словари.  
При помощи клиента обращаемся к базе `concordance`. Если эта база не существовала, она создастся при первой записи данных. Аналогично всё происходит с коллекциями.<br>

In [88]:
# Обращение к базе при помощи оператора квадратные скобки.
db = client['concordance']
# Обращение к базе при помощи оператора точка.
#db = client.concordance

Посмотрим что содержит в себе база.

In [89]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'concordance')

In [90]:
# Обращение к коллекции внутри базы при помощи оператора точка (но можно и при помощи квадратных скобок).
text_collection = db.nplus1texts
dictionary = db.dictionary
lemmas = db.lemmas
dbsents = db.sentences

vocabulary = db.vocabulary

В коллекции `nplus1texts` будем хранить адерса и идентификаторы текстов, в коллекции `dictionary` - словарь токенов, в `lemmas` - словарь лемм, наконец в `sentences` - разделение предложений на слова.

Для того, чтобы найти документ, необходимо использовать функцию `collection.find()`. Без параметров эта функция выдаст все документы.

In [91]:
from pprint import pprint

In [93]:
for t in text_collection.find()[:2]:
    pprint(t)

{'_id': ObjectId('616fcef4e3381e0ff188001c'),
 'art_info': {'art_date': '20 Янв. 2020',
              'art_time': '14:18',
              'author': 'Александр Войтюк',
              'difficulty': '2.3'},
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к '
             'Красной планете этим летом, успешно прошел заключительные '
             'термовакуумные испытания, имитировавшие\xa0условия марсианской '
             'среды, в которых предстоит работать роверу,\xa0сообщается на '
             'сайте ESA.Старт второго этапа российско-европейской программы '
             '«ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. '
             'Ракета-носитель «Протон» выведет в космос перелетный модуль, '
             'который доставит к Марсу десантный модуль, содержащий в себе '
             'автономную научную станцию «Казачок»\xa0и марсоход «Розалинд '
             'Франклин». Посадка на поверхность Красной планеты должна '
             'состояться 19 ма

Посмотрим и на другие коллекции.

In [94]:
for t in dictionary.find()[:2]:
    pprint(t)

{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff188001e'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c')],
 'freq': 1,
 'iniForm': 'марсоход',
 'token': 'Марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff1880021'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'розалинда',
 'token': 'Розалинд'}


In [95]:
for t in lemmas.find()[:2]:
    pprint(t)

{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff188001d'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 6,
 'iniForm': 'марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff1880020'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'розалинда'}


In [10]:
for t in dbsents.find()[:2]:
    pprint(t)

{'_id': ObjectId('616fcef4e3381e0ff188001f'),
 'position': 1,
 'sent_id': 1,
 'textId': ObjectId('616fcef4e3381e0ff188001c'),
 'wordFormId': ObjectId('616fcef4e3381e0ff188001e')}
{'_id': ObjectId('616fcef4e3381e0ff1880022'),
 'position': 2,
 'sent_id': 1,
 'textId': ObjectId('616fcef4e3381e0ff188001c'),
 'wordFormId': ObjectId('616fcef4e3381e0ff1880021')}


Чтобы поиск производился быстрее можно создать индексы.

In [11]:
# Добавляем индекс в коллекцию lemmas по полям iniForm и POS, оба по возрастанию.
# Заодно запрещаем добавлять повторяющиеся сочетания.
lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)
dictionary.create_index([('token', pymongo.ASCENDING), ('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

'token_1_iniForm_1_POS_1'

Или удалить их, если они больше не нужны.

In [7]:
# Для пробы чуть позже удалим индекс и посмотрим на изменение скорости работы.
lemmas.drop_index('iniForm_1_POS_1')

В качестве параметров функции передается документ (в случае Python - словарь) с полями и их значениями. 

Если значение должно отвечать не равенству, а другому условию, передается составное значение в виде словаря, ключем у которого будет обозначение оператора, а значением - значение с которым будет проводиться сравнение. Таких пар можно передать несколько.

In [97]:
# (freq _какое_ 2) _и_ (token _какое_ 'может')
#for l in dictionary.find({"freq": 2, 'token': 'может'}):
for l in dictionary.find({'token': 'ученые', 'freq': 15}):
    pprint(l)


{'POS': 'NOUN',
 '_id': ObjectId('616fcef5e3381e0ff18802ae'),
 'docs': [ObjectId('616fcef5e3381e0ff188026b'),
          ObjectId('616fcef6e3381e0ff18807da'),
          ObjectId('616fcef8e3381e0ff1880c75'),
          ObjectId('616fcefae3381e0ff1880ee4'),
          ObjectId('616fcefce3381e0ff18813a0'),
          ObjectId('616fcefee3381e0ff1881767')],
 'freq': 15,
 'iniForm': 'учёный',
 'token': 'ученые'}


In [98]:
# freq _какое_ меньше _чего_ 5
for l in lemmas.find({"freq":{"$lt": 5}})[:5]:
    pprint(l)

print("----")
# freq _какое_ меньше _чего_ 5
for l in dictionary.find({"freq":{"$lt": 5}})[:5]:
    pprint(l)
    
print("----")
# freq _какое_ ((больше _чего_ 5) _и_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5, "$lt": 9}})[:5]:
    pprint(l)


{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff1880020'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'розалинда'}
{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff1880023'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'франклин'}
{'POS': 'INFN',
 '_id': ObjectId('616fcef4e3381e0ff188002c'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c')],
 'freq': 1,
 'iniForm': 'отправиться'}
{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff188003b'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('616fceffe3381e0ff18819c7'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'лето'}
{'POS': 'ADVB',
 '_id': ObjectId('616fcef4e3381e0ff188003e'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'успешно'}
----
{'POS': 'NOUN',
 '_id

In [102]:
text_collection.find({"_id":ObjectId('616fcefce3381e0ff188130d')})[0]

{'_id': ObjectId('616fcefce3381e0ff188130d'),
 'text_url': 'https://nplus1.ru/news/2018/03/20/abel-prize-2018',
 'text_name': 'Абелевская премия присуждена за открытие связи между теорией чисел и теорией представлений',
 'art_text': 'Норвежская академия наук объявила лауреата Абелевской премии 2018 года. Им стал канадский математик\xa0Роберт Ленглендс. Премия присуждена «за дальновидную программу, соединяющую теорию представлений и теорию чисел».\xa0Абелевскую премию часто называют «Нобелевской премией по математике». В отличие от, например, Филдсовской медали она вручается каждый год. Размер премии — 6 миллионов крон (около 45 миллионов рублей). Церемония награждения пройдет 22 мая 2018 года в Университете Аула, Осло. Вручать награду будет лично король Норвегии Харальд V.\n    \n        \n                        \n                \n                    \n                        \n                    ',
 'art_info': {'art_date': '20 Март 2018',
  'art_time': '14:15',
  'difficulty': '6.

In [103]:
# Можно записать и так.
# (freq _какое_ (больше _чего_ 5)() _и_ (freq _какое_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5}, "freq": {"$lt": 9}})[:5]:
    pprint(l)

{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff188001e'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c')],
 'freq': 1,
 'iniForm': 'марсоход',
 'token': 'Марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff1880021'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'розалинда',
 'token': 'Розалинд'}
{'POS': 'NOUN',
 '_id': ObjectId('616fcef4e3381e0ff1880024'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('61ba0f43e48154e25857f07d')],
 'freq': 3,
 'iniForm': 'франклин',
 'token': 'Франклин'}
{'POS': 'ADJS',
 '_id': ObjectId('616fcef4e3381e0ff188002a'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c'),
          ObjectId('616fcef8e3381e0ff1880c75')],
 'freq': 3,
 'iniForm': 'должный',
 'token': 'должен'}
{'POS': 'INFN',
 '_id': ObjectId('616fcef4e3381e0ff188002d'),
 'docs': [ObjectId('616fcef4e3381e0ff188001c')],
 'freq': 1,
 'iniForm': 'отправиться',
 'token': 'отправиться'}


In [105]:
res1 = lemmas.find({"freq":{"$gt": 5}})
res1[0]['iniForm'], type(res1[0])

('марсоход', dict)

Попробуем найти все документы со сложностью равной 2.3 (статья про ЭкзоМарс).

In [108]:
for l in text_collection.find({'art_info': {'difficulty': '6.8'}}):
    print(l)

In [107]:
text_collection.find({'art_info': {'art_date': '20 Март 2018',
  'art_time': '14:15',
  'difficulty': '6.8',
  'author': 'Владимир Королев'}})[0]

{'_id': ObjectId('616fcefce3381e0ff188130d'),
 'text_url': 'https://nplus1.ru/news/2018/03/20/abel-prize-2018',
 'text_name': 'Абелевская премия присуждена за открытие связи между теорией чисел и теорией представлений',
 'art_text': 'Норвежская академия наук объявила лауреата Абелевской премии 2018 года. Им стал канадский математик\xa0Роберт Ленглендс. Премия присуждена «за дальновидную программу, соединяющую теорию представлений и теорию чисел».\xa0Абелевскую премию часто называют «Нобелевской премией по математике». В отличие от, например, Филдсовской медали она вручается каждый год. Размер премии — 6 миллионов крон (около 45 миллионов рублей). Церемония награждения пройдет 22 мая 2018 года в Университете Аула, Осло. Вручать награду будет лично король Норвегии Харальд V.\n    \n        \n                        \n                \n                    \n                        \n                    ',
 'art_info': {'art_date': '20 Март 2018',
  'art_time': '14:15',
  'difficulty': '6.

Ничего не найдено, так как такая запись предполагает полное совпадение, то есть поиск поддокумента с единственным полем и этим значением. Если полей будет несколько, роль будет играть даже порядок следования полей в базе!

То, что мы хотели сделать,нужно сделать вот так.

In [109]:
for l in text_collection.find({'art_info.difficulty': '2.3'}):
    pprint(l)

{'_id': ObjectId('616fcef4e3381e0ff188001c'),
 'art_info': {'art_date': '20 Янв. 2020',
              'art_time': '14:18',
              'author': 'Александр Войтюк',
              'difficulty': '2.3'},
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к '
             'Красной планете этим летом, успешно прошел заключительные '
             'термовакуумные испытания, имитировавшие\xa0условия марсианской '
             'среды, в которых предстоит работать роверу,\xa0сообщается на '
             'сайте ESA.Старт второго этапа российско-европейской программы '
             '«ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. '
             'Ракета-носитель «Протон» выведет в космос перелетный модуль, '
             'который доставит к Марсу десантный модуль, содержащий в себе '
             'автономную научную станцию «Казачок»\xa0и марсоход «Розалинд '
             'Франклин». Посадка на поверхность Красной планеты должна '
             'состояться 19 ма

Можно выбирать не все поля, а только те, которые нам необходимы.

In [111]:
for t in text_collection.find(projection=['art_info.difficulty','text_name'])[:5]:
    pprint(t)

{'_id': ObjectId('616fcef4e3381e0ff188001c'),
 'art_info': {'difficulty': '2.3'},
 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания'}
{'_id': ObjectId('616fcef5e3381e0ff188026b'),
 'art_info': {'difficulty': '4.2'},
 'text_name': 'Физики заперли свет в нанорезонаторе на рекордно долгое время'}
{'_id': ObjectId('616fcef6e3381e0ff1880558'),
 'art_info': {'difficulty': '1.7'},
 'text_name': 'Пилотируемый полет Crew Dragon к МКС состоится в первой '
              'половине 2020 года'}
{'_id': ObjectId('616fcef6e3381e0ff18807da'),
 'art_info': {'difficulty': '3.9'},
 'text_name': 'Сон помог вознаграждению улучшить зрительное обучение'}
{'_id': ObjectId('616fcef8e3381e0ff1880c75'),
 'art_info': {'difficulty': '1.9'},
 'text_name': 'Машинное обучение помогло распознать человека по танцу'}


Ключевое слово `$or` и другие операторы позволяют применять логические свзки помимо логического И.

In [154]:
for d in dictionary.find({'$or':[{'token': 'объединяет'}, {'freq': 3}]})[:5]:
    print(d)

{'_id': ObjectId('616fcef4e3381e0ff1880021'), 'token': 'Розалинд', 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('61ba0f43e48154e25857f07d')]}
{'_id': ObjectId('616fcef4e3381e0ff1880024'), 'token': 'Франклин', 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('61ba0f43e48154e25857f07d')]}
{'_id': ObjectId('616fcef4e3381e0ff188002a'), 'token': 'должен', 'iniForm': 'должный', 'POS': 'ADJS', 'freq': 3, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcef8e3381e0ff1880c75')]}
{'_id': ObjectId('616fcef4e3381e0ff188003f'), 'token': 'успешно', 'iniForm': 'успешно', 'POS': 'ADVB', 'freq': 3, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('61ba0f43e48154e25857f07d')]}
{'_id': ObjectId('616fcef4e3381e0ff1880057'), 'token': 'среды', 'iniForm': 'среда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcef5e3381e0ff188026b')]}


Для поиска в массивах одного значения можно просто проводить поиск, БД сама будет искать объекты в массиве.

Если мы ищем несколько значений в массиве, необходимо использовать соответствующие предикаты: `$in` для поиска вхождения одного из элементов, `$all` - поиска всех элементов, и т.д.

Обратите внимание, для поиска мы используем не строку, а объект `ObjectId`, импортированный из библиотеки `bson`.

Помимо этого, во втором цикле в поле projection мы передаем не только какие поля надо выводить, но и какие поля выводить не надо. Так, поле \_id, хранящее идентификатор документа, по умолчанию должно выводиться. В примере ниже оно подавляется при помощи `"_id":False`.

In [23]:
for l in dictionary.find({'token': 'ученые'}):
    print(l['docs'][0])
    pprint(l)
    for d in dictionary.find({"docs": {"$in":[l['docs'][0]]}})[:5]:
        print(d)
    

616fcef5e3381e0ff188026b
{'POS': 'NOUN',
 '_id': ObjectId('616fcef5e3381e0ff18802ae'),
 'docs': [ObjectId('616fcef5e3381e0ff188026b'),
          ObjectId('616fcef6e3381e0ff18807da'),
          ObjectId('616fcef8e3381e0ff1880c75'),
          ObjectId('616fcefae3381e0ff1880ee4'),
          ObjectId('616fcefce3381e0ff18813a0'),
          ObjectId('616fcefee3381e0ff1881767')],
 'freq': 15,
 'iniForm': 'учёный',
 'token': 'ученые'}
{'_id': ObjectId('616fcef4e3381e0ff1880030'), 'token': 'к', 'iniForm': 'к', 'POS': 'PREP', 'freq': 25, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcef5e3381e0ff188026b'), ObjectId('616fcef6e3381e0ff1880558'), ObjectId('616fcef6e3381e0ff18807da'), ObjectId('616fcef9e3381e0ff1880e4b'), ObjectId('616fcefce3381e0ff18813a0'), ObjectId('616fcefee3381e0ff1881767'), ObjectId('616fceffe3381e0ff18819c7'), ObjectId('61ba0f43e48154e25857f07d'), ObjectId('61ba275ef8c2a15d447e6fb6')]}
{'_id': ObjectId('616fcef4e3381e0ff1880057'), 'token': 'среды', 'iniForm': '

In [112]:
for d in dictionary.find({"docs": ObjectId('616fcef5e3381e0ff188026b')})[:3]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$in": [ObjectId('616fcef5e3381e0ff188026b')]}})[:3]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$all": [ObjectId('616fcef5e3381e0ff188026b'), ObjectId('616fcef9e3381e0ff1880e4b')]}},
                         projection={"_id":False, "token":True, "docs":True})[:3]:
    print(d)    

{'_id': ObjectId('616fcef4e3381e0ff1880030'), 'token': 'к', 'iniForm': 'к', 'POS': 'PREP', 'freq': 31, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcef5e3381e0ff188026b'), ObjectId('616fcef6e3381e0ff1880558'), ObjectId('616fcef6e3381e0ff18807da'), ObjectId('616fcef9e3381e0ff1880e4b'), ObjectId('616fcefce3381e0ff18813a0'), ObjectId('616fcefee3381e0ff1881767'), ObjectId('616fceffe3381e0ff18819c7'), ObjectId('61ba0f43e48154e25857f07d'), ObjectId('61ba275ef8c2a15d447e6fb6'), ObjectId('61f257d9cc4b9bf056b637c1'), ObjectId('61f25e27cc4b9bf056b63d4c'), ObjectId('61f25e44cc4b9bf056b63ef7'), ObjectId('61f25ea9cc4b9bf056b643cf')]}
{'_id': ObjectId('616fcef4e3381e0ff1880057'), 'token': 'среды', 'iniForm': 'среда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcef5e3381e0ff188026b')]}
{'_id': ObjectId('616fcef4e3381e0ff188005a'), 'token': 'в', 'iniForm': 'в', 'POS': 'PREP', 'freq': 227, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId

In [113]:
# А это мы просто смотрим сколько текстов добавили в базу.
# В качестве параметра передается условие, считаются только удовлетворяющие ему документы.
# В данном случае нам нужны все.
print(text_collection.count_documents({}))

# Ищем количество слов с частотой встречаемости от 6 до 9.
print(lemmas.count_documents({"freq":{"$gt": 5, "$lt": 10}}))

# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим не выводить идентификатор и отсортировать всё по частоте.
for l in lemmas.find({"freq":{"$gt": 5, "$lt": 10}}, {"_id":False}).sort("iniForm")[:5]:
    print(l)
print("----")
# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим вывести токен и частоту, но не выводить идентификатор и отсортировать всё по частоте.
for l in dictionary.find({"freq":{"$gt": 5}}, {"token": True, "freq":True, "_id":False}, 
                         skip=25, limit=5).sort("freq"):
    print(l)


22
99
{'iniForm': 'американский', 'POS': 'ADJF', 'freq': 9, 'docs': [ObjectId('616fcef6e3381e0ff1880558'), ObjectId('616fcef6e3381e0ff18807da'), ObjectId('616fcefce3381e0ff18813a0'), ObjectId('61ba0f43e48154e25857f07d'), ObjectId('61ba275ef8c2a15d447e6fb6'), ObjectId('61f257a2cc4b9bf056b637c0'), ObjectId('61f257d9cc4b9bf056b637c1'), ObjectId('61f25e27cc4b9bf056b63d4c')]}
{'iniForm': 'анализ', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('616fcef6e3381e0ff1880558'), ObjectId('616fceffe3381e0ff18819c7'), ObjectId('61f25e44cc4b9bf056b63ef7'), ObjectId('61f25ea9cc4b9bf056b643cf')]}
{'iniForm': 'бластер', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('61f25e44cc4b9bf056b63ef7'), ObjectId('61f25ea9cc4b9bf056b643cf')]}
{'iniForm': 'больший', 'POS': 'ADJF', 'freq': 7, 'docs': [ObjectId('616fcefae3381e0ff1880ee4'), ObjectId('616fcefbe3381e0ff188112d'), ObjectId('616fcefce3381e0ff18813a0'), ObjectId('616fcefee3381e0ff1881767'), ObjectId('616fceffe3381e0ff18819c7')]}
{'iniForm': 'версия', 'POS': 'NO

In [117]:
for d in dictionary.find({"docs": {"$size": 1}}, limit=5):
    print(d)


{'_id': ObjectId('616fcef4e3381e0ff188001e'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('616fcef4e3381e0ff188001c')]}
{'_id': ObjectId('616fcef4e3381e0ff188002d'), 'token': 'отправиться', 'iniForm': 'отправиться', 'POS': 'INFN', 'freq': 1, 'docs': [ObjectId('616fcef4e3381e0ff188001c')]}
{'_id': ObjectId('616fcef4e3381e0ff1880036'), 'token': 'планете', 'iniForm': 'планета', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('616fcef4e3381e0ff188001c')]}
{'_id': ObjectId('616fcef4e3381e0ff1880039'), 'token': 'этим', 'iniForm': 'это', 'POS': 'NPRO', 'freq': 1, 'docs': [ObjectId('616fcef4e3381e0ff188001c')]}
{'_id': ObjectId('616fcef4e3381e0ff1880042'), 'token': 'прошел', 'iniForm': 'пройти', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('616fcef4e3381e0ff188001c')]}


Теперь напишем функцию для того, чтобы класть статью в БД. используем для этого функции:
- insert_one (insert, insert_many) - добавляет запись (несколько записей) в выбранную коллекцию.
- find_one_and_update - найти и обновить (аналог update).

В качестве параметра во все функции добавления передается документ (словарь), ключи которого содержат названия полей, а значения - значения этих полей. Значение поля само может являться документом. При поиске часть полей может принимать особые значения, например, `$ne` - не равно, `$size` - смотрим на размер массива, `$inc` - увеличить значение поля на заданную величину и т.д. Более подробно с соответствующими ключами можно ознакомиться <a href="https://metanit.com/nosql/mongodb/" >здесь</a>.

In [118]:
# Создаем морфоанализатор.
morph = pymorphy2.MorphAnalyzer()

In [54]:
class NPlus1Article:
    def __init__(self):
        self.time = ""
        self.date = ""
        self.rubr = ""
        self.diff = ""
        self.author = ""
        self.head = ""
        self.text = ""
        
    def __repr__(self) -> str:
        return f"{self.head}, {self.author}, {self.date}, {self.rubr}, {self.text}"

def getArticleTextNPlus1(adr):
    page = requests.get(adr)
    #print(r.text)
    art=NPlus1Article()
    tree = html.fromstring(page.text)
    art.head = tree.xpath(".//h1")[0].text_content()
    dt = tree.xpath(".//time//span")
    art.date = dt[1].text_content()
    art.time = dt[0].text_content()
    art.rubr = tree.xpath(".//a[@data-rubric]")[0].text_content()
    art.diff = tree.xpath(".//span[@class='difficult-value']")[0].text_content()
    art.author = tree.xpath(".//meta[@name='mediator_author']")[0].get("content")
    art.text = '\n'.join([p.text_content() for p in tree.xpath(".//article//p")[3:-2]])
    return art

def putNPlus1ArticleInMongo(art_url):
    # Загружаем текст статьи и другие ее части.
    art = getArticleTextNPlus1(art_url) 

    # Добавляем запись с информацией о статье.
    # inserted_id позволяет сразу получить идентификатор записи, чтобы потом ссылаться на него там, где это необходимо.
    a_text = {"text_url": art_url, "text_name": art.head, "art_text": art.text, 
              "art_info": {"art_date": art.date, "art_time": art.time, 
                           "difficulty": art.diff, "author": art.author}}
    text_id = text_collection.insert_one(a_text).inserted_id
    print(text_id)

    # Выделяем предложения (просто по точке с пробелом или в конце строки!!!).
    sents = re.split("\.\s|\.$", art.text)
    sent_num = 1
    # Загружаем предложения в базу.
    for s in tqdm(sents):
        # Выделяем слова (просто как группы русских букв!!!).
        words = re.findall("([А-Яа-я]+(\-[А-Яа-я]+)?)", s)
        posit = 1
        # Загрузка слов из предложений.
        for w in words:
            wf = morph.parse(w[0])
            # Провели морфологический анализ и теперь добавляем документ с двумя полями: нач. форма и часть речи.
            # Сперва проверяем есть ли там уже такая запись.
            # Смотрим сколько таких слов нашлось. Если ни одного - надо добавлять.
            if lemmas.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = lemmas.insert_one({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS, 
                                              "freq": 1, "docs":[text_id]}).inserted_id
            # А если такое слово уже было, то обновляем частоту встречаемости и в каком документе оно встретиось.
            else:
                inis = lemmas.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                lemmas.find_one_and_update({"_id": lemma_id}, 
                                           {"$inc": {"freq": 1}, 
                                            "$addToSet": {"docs": text_id}})
                # Вот таким образом можно добавлять номер документа столько раз, сколько в нем встретилось слово.
                #lemmas.find_one_and_update({"_id": lemma_id}, {"$push": {"docs": text_id}}) 

            # Повторяем операцию для токенов.
            if dictionary.count_documents({"token":w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                wf_id = dictionary.insert_one({"token":w[0], 
                                               "iniForm": wf[0].normal_form, 
                                               "POS": wf[0].tag.POS, 
                                               "freq": 1, 
                                               "docs": [text_id]}).inserted_id
            else:
#                 wrdf = dictionary.find({"token": w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
#                 wf_id = wrdf[0]["_id"]
#                 dictionary.find_one_and_update({"_id": wf_id}, {"$inc":  {"freq": 1}}) 
#                 dictionary.find_one_and_update({"_id": wf_id}, {"$addToSet": {"docs": text_id}}) 
                dictionary.find_one_and_update({"token": w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS},
                                               {"$inc":  {"freq": 1}, "$addToSet": {"docs": text_id}}) 
  
            # Добавляем номер предложения, идентификатор словоформы из словаря, позицию слова, из какого оно текста.
            dbsents.insert_one({"sent_id": sent_num, "wordFormId":wf_id, "position": posit, "textId": text_id})
            posit += 1  
        sent_num += 1

In [72]:
class NPlus1Article:
    def __init__(self):
        self.time = ""
        self.date = ""
        self.rubr = ""
        self.diff = ""
        self.author = ""
        self.head = ""
        self.text = ""
        
    def __repr__(self) -> str:
        return f"{self.head}, {self.author}, {self.date}, {self.rubr}, {self.text}"

def getArticleTextNPlus1(adr):
    page = requests.get(adr)
    #print(r.text)
    art = NPlus1Article()
    tree = html.fromstring(page.text)
    art.head = tree.xpath(".//h1")[0].text_content()
    dt = tree.xpath(".//time//span")
    art.date = dt[1].text_content()
    art.time = dt[0].text_content()
    art.rubr = tree.xpath(".//a[@data-rubric]")[0].text_content()
    art.diff = tree.xpath(".//span[@class='difficult-value']")[0].text_content()
    art.author = tree.xpath(".//meta[@name='mediator_author']")[0].get("content")
    art.text = '\n'.join([p.text_content() for p in tree.xpath(".//article//p")[3:-2]])
    return art

def putNPlus1ArticleInMongo(art_url):
    # Загружаем текст статьи и другие ее части.
    art = getArticleTextNPlus1(art_url) 

    # Добавляем запись с информацией о статье.
    # inserted_id позволяет сразу получить идентификатор записи, чтобы потом ссылаться на него там, где это необходимо.
    a_text = {"text_url": art_url, "text_name": art.head, "art_text": art.text, 
              "art_info": {"art_date": art.date, "art_time": art.time, 
                           "difficulty": art.diff, "author": art.author}}
    text_id = text_collection.insert_one(a_text).inserted_id
    print(text_id)

    # Выделяем предложения (просто по точке с пробелом или в конце строки!!!).
    sents = re.split("\.\s|\.$", art.text)
    sent_num = 1
    # Загружаем предложения в базу.
    for s in tqdm(sents):
        # Выделяем слова (просто как группы русских букв!!!).
        words = re.findall("([А-Яа-я]+(\-[А-Яа-я]+)?)", s)
        posit = 1
        # Загрузка слов из предложений.
        for w in words:
            wf = morph.parse(w[0])
            # Провели морфологический анализ и теперь добавляем документ с двумя полями: нач. форма и часть речи.
            # Сперва проверяем есть ли там уже такая запись.
            # Смотрим сколько таких слов нашлось. Если ни одного - надо добавлять.
            if lemmas.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = lemmas.insert_one({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS, 
                                              "freq": 1, "docs":[text_id]}).inserted_id
            # А если такое слово уже было, то обновляем частоту встречаемости и в каком документе оно встретиось.
            else:
                inis = lemmas.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                lemmas.find_one_and_update({"_id": lemma_id}, 
                                           {"$inc": {"freq": 1}, 
                                            "$addToSet": {"docs": text_id}})
                # Вот таким образом можно добавлять номер документа столько раз, сколько в нем встретилось слово.
                #lemmas.find_one_and_update({"_id": lemma_id}, {"$push": {"docs": text_id}}) 

            # Повторяем операцию для токенов.
            if dictionary.count_documents({"token":w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                wf_id = dictionary.insert_one({"token":w[0], 
                                               "iniForm": wf[0].normal_form, 
                                               "POS": wf[0].tag.POS, 
                                               "freq": 1, 
                                               "docs": [text_id]}).inserted_id
            else:
                wrdf = dictionary.find({"token": w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                wf_id=wrdf[0]["_id"]
                dictionary.find_one_and_update({"_id": wf_id}, {"$inc":  {"freq": 1}}) 
                dictionary.find_one_and_update({"_id": wf_id}, {"$addToSet": {"docs": text_id}}) 
  


            if vocabulary.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = vocabulary.insert_one({"iniForm": wf[0].normal_form, 
                                                  "POS": wf[0].tag.POS, 
                                                  "freq": 1, 
                                                  "tokens": [{"token": w[0].lower(), "freq": 1, 
                                                              "docs": [text_id]}]}
                                                ).inserted_id
            else:
                inis = vocabulary.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                vocabulary.find_one_and_update({"_id": lemma_id}, 
                                               {"$inc": {"freq": 1}})#, 
#                                                 "$addToSet": {"token.docs": text_id}})

                if vocabulary.count_documents({"_id": lemma_id,
                                               "tokens.$.token": w[0].lower()
                                              }
                                             ) == 0:
                    _ = vocabulary.find_one_and_update({"_id": lemma_id}, 
                                                       {"$addToSet": {"tokens": {"token": w[0].lower(), "freq": 1, 
                                                        "docs": [text_id]}}}
                                                      )
                else:
                    _ = vocabulary.find_one_and_update({"_id": lemma_id, "tokens.token": w[0].lower()},
                                                       {"$inc": {"tokens.freq": 1}}
                                                      )


            # Добавляем номер предложения, идентификатор словоформы из словаря, позицию слова, из какого оно текста.
            dbsents.insert_one({"sent_id": sent_num, "wordFormId":wf_id, "position": posit, "textId": text_id})
            posit += 1  
        sent_num += 1

In [119]:
# Добавляем статью  в базу.
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/subwavelength-resonators")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/crew-dragon-first-crewed-flight")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/visual-perception")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/dancing-recognition")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/27/juno-pictures")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/04/12/alignment-of-jets")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/29/Ultra-Deep-Survey")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/balance")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/abel-prize-2018")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/10/12/srb-electrode")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/02/19/friction-on-ice")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/04/07/illex-coindetii")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/12/15/parachute-exomars-2022")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/26/flw")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/26/cure-rare")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/26/cheap-blaster-sound")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/27/cosmology-with-one-galaxy")

61f27f26cc4b9bf056b645f8


100%|███████████████████████████████████████████| 33/33 [00:04<00:00,  7.31it/s]


In [120]:
# for l in vocabulary.find({"iniForm": 'американский', "POS": 'ADJF'}):
for l in vocabulary.find({'freq':{'$gt':1}}):
    pprint(l)

{'POS': 'ADJF',
 '_id': ObjectId('61f25d91cc4b9bf056b63a60'),
 'freq': 2,
 'iniForm': 'французский',
 'tokens': [{'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'французские'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25d91cc4b9bf056b63a64'),
 'freq': 5,
 'iniForm': 'врач',
 'tokens': [{'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'врачи'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25d91cc4b9bf056b63a6d'),
 'freq': 3,
 'iniForm': 'состояние',
 'tokens': [{'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'состояние'},
            {'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,
             'token': 'состояния'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25d91cc4b9bf056b63a71'),
 'freq': 8,
 'iniForm': 'ребёнок',
 'tokens': [{'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'детей'},
       

            {'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'новых'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'новых'}]}
{'POS': 'ADJF',
 '_id': ObjectId('61f25e27cc4b9bf056b63e65'),
 'freq': 8,
 'iniForm': 'тот',
 'tokens': [{'docs': [ObjectId('61f25e27cc4b9bf056b63d4c')],
             'freq': 1,
             'token': 'том'},
            {'docs': [ObjectId('61f25e27cc4b9bf056b63d4c')],
             'freq': 1,
             'token': 'того'},
            {'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'том'},
            {'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'того'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'том'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
          

             'token': 'дешевизна'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'дешевизна'}]}
{'POS': 'ADJF',
 '_id': ObjectId('61f25e44cc4b9bf056b63fd1'),
 'freq': 2,
 'iniForm': 'низкий',
 'tokens': [{'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'низкий'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'низкий'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25e44cc4b9bf056b63fd5'),
 'freq': 2,
 'iniForm': 'порог',
 'tokens': [{'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'порог'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'порог'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25e44cc4b9bf056b63fd9'),
 'freq': 2,
 'iniForm': 'навык',
 'tokens': [{'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,

            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'именно'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25e45cc4b9bf056b641d5'),
 'freq': 2,
 'iniForm': 'способ',
 'tokens': [{'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'способом'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'способом'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f25e45cc4b9bf056b641d9'),
 'freq': 2,
 'iniForm': 'бен',
 'tokens': [{'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'бен'},
            {'docs': [ObjectId('61f25ea9cc4b9bf056b643cf')],
             'freq': 1,
             'token': 'бен'}]}
{'POS': 'PRCL',
 '_id': ObjectId('61f25e45cc4b9bf056b641dd'),
 'freq': 2,
 'iniForm': 'б',
 'tokens': [{'docs': [ObjectId('61f25e44cc4b9bf056b63ef7')],
             'freq': 1,
             'token': 'б'},
            {'do

 'tokens': [{'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,
             'token': 'вселенной'},
            {'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,
             'token': 'вселенную'}]}
{'POS': 'PRTF',
 '_id': ObjectId('61f27f27cc4b9bf056b646aa'),
 'freq': 2,
 'iniForm': 'фиксировать',
 'tokens': [{'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,
             'token': 'фиксированной'},
            {'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,
             'token': 'фиксированным'}]}
{'POS': 'ADJF',
 '_id': ObjectId('61f27f27cc4b9bf056b646af'),
 'freq': 2,
 'iniForm': 'барионный',
 'tokens': [{'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,
             'token': 'барионного'}]}
{'POS': 'NOUN',
 '_id': ObjectId('61f27f27cc4b9bf056b646b2'),
 'freq': 3,
 'iniForm': 'вещество',
 'tokens': [{'docs': [ObjectId('61f27f26cc4b9bf056b645f8')],
             'freq': 1,


In [123]:
for l in vocabulary.find({"_id": ObjectId('61f25d91cc4b9bf056b63a71'), "tokens.token": 'детей'}):#,
#                                                        {"$inc": {"tokens.$.freq": 1}}
#                                                       )
    pprint(l)

{'POS': 'NOUN',
 '_id': ObjectId('61f25d91cc4b9bf056b63a71'),
 'freq': 8,
 'iniForm': 'ребёнок',
 'tokens': [{'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'детей'},
            {'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'детям'},
            {'docs': [ObjectId('61f25d91cc4b9bf056b63a5e')],
             'freq': 1,
             'token': 'ребенку'}]}


In [48]:
# Вот так можно удалить всё в базе. Если написать условие, то не всё. 
# Экспериментировать не будем, да?
text_collection.delete_many({})
dictionary.delete_many({})
lemmas.delete_many({})
dbsents.delete_many({})
#for l in dbsents.find():
# print(l)


In [125]:
# Так можно посмотреть все записи, у которых в токене записан список ровно из пяти элементов.
#for l in dictionary.find({"docs":{"$size":5}})[:5]:
for l in dictionary.find({"docs":{"$size":4}})[:5]:
    print(l)
print("=====")
# Так можно посмотреть все записи, у которых есть поле docs.
# for l in lemmas.find({"docs":{"$exists":True}})[:5]:
for l in text_collection.find({"coord":{"$exists":True}})[:5]:
    print(l)

{'_id': ObjectId('616fcef4e3381e0ff188006e'), 'token': 'сайте', 'iniForm': 'сайт', 'POS': 'NOUN', 'freq': 4, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcef9e3381e0ff1880e4b'), ObjectId('616fcefae3381e0ff1880ee4'), ObjectId('61ba0f43e48154e25857f07d')]}
{'_id': ObjectId('616fcef4e3381e0ff18801b6'), 'token': 'составляет', 'iniForm': 'составлять', 'POS': 'VERB', 'freq': 4, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcefbe3381e0ff188112d'), ObjectId('616fcefce3381e0ff18813a0'), ObjectId('616fcefee3381e0ff1881767')]}
{'_id': ObjectId('616fcef4e3381e0ff18801e1'), 'token': 'градусов', 'iniForm': 'градус', 'POS': 'NOUN', 'freq': 8, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fcefbe3381e0ff18810a6'), ObjectId('616fcefce3381e0ff18813a0'), ObjectId('616fcefee3381e0ff1881767')]}
{'_id': ObjectId('616fcef4e3381e0ff188020f'), 'token': 'всех', 'iniForm': 'весь', 'POS': 'ADJF', 'freq': 4, 'docs': [ObjectId('616fcef4e3381e0ff188001c'), ObjectId('616fce

При помощи ключевого слова `$regex' можно искать в базе строки, отвечающие регулярным выражениям.

In [126]:
# iniForm _какое_ описывается_регуляркой _какой_ ^и.+
for d in lemmas.find({'iniForm':{'$regex':'^и.+'}})[:5]:
    print(d)
print("====")    
for d in dictionary.find({'token':{'$regex':'^[А-Я].+'}})[:5]:
    print(d)

{'_id': ObjectId('616fcef8e3381e0ff1880c72'), 'iniForm': 'ивтушок', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('616fcef6e3381e0ff18807da')]}
{'_id': ObjectId('616fcef7e3381e0ff1880834'), 'iniForm': 'играть', 'POS': 'VERB', 'freq': 2, 'docs': [ObjectId('616fcef6e3381e0ff18807da')]}
{'_id': ObjectId('61ba275ef8c2a15d447e6ff6'), 'iniForm': 'идеальный', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('61ba275ef8c2a15d447e6fb6'), ObjectId('61f27f26cc4b9bf056b645f8')]}
{'_id': ObjectId('61f27f28cc4b9bf056b64891'), 'iniForm': 'идентифицироваться', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('61f27f26cc4b9bf056b645f8')]}
{'_id': ObjectId('616fcef6e3381e0ff188076b'), 'iniForm': 'идея', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('616fcef6e3381e0ff1880558'), ObjectId('616fceffe3381e0ff18819c7')]}
====
{'_id': ObjectId('616fcefce3381e0ff188131d'), 'token': 'Абелевской', 'iniForm': 'абелёвский', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('616fcefce3381e0ff188130d')], 'found': 'true'}
{'_id': ObjectI

Обратите внимание, что при обновлении заменяетя весь документ целиком. Чтобы избежать этого следует использовать ключевое слово `$set`, которое показывает, что обновляется значение только одного поля.

In [131]:
dictionary.insert_one({'token':'Ъфыва', 'found':'false'})

In [133]:
dictionary.find_one_and_update({'token':{'$regex':'^Ъ.+'}, 'found':{"$exists":True}}, {'$set': {'found':'true'}})
# Просто посмотрим на документы
for d in dictionary.find({'token':{'$regex':'^Ъ.+'}, 'found':{"$exists":True}}):
    pprint(d)

{'_id': ObjectId('61f2854ecc4b9bf056b64c07'), 'found': 'true', 'token': 'Ъфыва'}


In [130]:
# Просто посмотрим на документы
for d in dictionary.find({'token':{'$regex':'^И.+'}, 'found':'$exists'}):
    print(d)

In [54]:
# А теперь найдем первое попавшееся слово, начинающееся с заглавное И и добавим ему новое поле found
dictionary.find_one_and_update({'token':{'$regex':'^И.+'}, 'found':'false'}, {'$set': {'found':'true'}})

In [129]:
for r in dictionary.find({'token': 'Изначально'}):
    print(r)

{'_id': ObjectId('616fcef6e3381e0ff1880720'), 'token': 'Изначально', 'iniForm': 'изначально', 'POS': 'ADVB', 'freq': 1, 'docs': [ObjectId('616fcef6e3381e0ff1880558')]}


Ничего не получилось, так как поле found отсутствует у всех записей. Попробуем теперь найти первую запись, у которой нет такого поля: 'found':{"$exists":False}} .

In [56]:
dictionary.find_one_and_update({'token':{'$regex':'^А.+'}, 'found':{"$exists":False}}, {'$set': {'found':'true'}})

{'_id': ObjectId('616fcefce3381e0ff1881349'),
 'token': 'Абелевскую',
 'iniForm': 'абелёвский',
 'POS': 'ADJF',
 'freq': 1,
 'docs': [ObjectId('616fcefce3381e0ff188130d')]}

In [134]:
# Попробуем найти все документы с полем found.
for d in dictionary.find({'token':{'$regex':'^А.+'}, 'found':{"$exists":True}}):
    print(d)

{'_id': ObjectId('616fcefce3381e0ff188131d'), 'token': 'Абелевской', 'iniForm': 'абелёвский', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('616fcefce3381e0ff188130d')], 'found': 'true'}
{'_id': ObjectId('616fcefce3381e0ff1881349'), 'token': 'Абелевскую', 'iniForm': 'абелёвский', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('616fcefce3381e0ff188130d')], 'found': 'true'}
{'_id': ObjectId('616fcef5e3381e0ff18803c5'), 'token': 'Австралии', 'iniForm': 'австралия', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('616fcef5e3381e0ff188026b')], 'found': 'true'}


<font color='#FF0000'>db.collection.updateMany(filter, update, options)</font>

#### Важное замечание

Вся структура нашей базы данных отдает реляционностью. Никто не мешал нам сделать документ, вложенный в словарь начальных форм и хранить в нем все формы данного слова. Тогда если что, не надо будет делать запросы, в которых надо делать пересечение двух коллекций.

{"iniForm":"абелёвский", 'POS': 'ADJF', "forms":[{'token': 'Абелевской', "tags":"род ед муж"}, {'token': 'Абелевский', "tags":"им ед муж"}]}

## Работа с географическими данными

MongoDB позволяет работать с географическими данными. Основой является [GEO JSON](https://docs.mongodb.com/manual/reference/geojson/), позволяющий обрабатывать разные виды геолокаций.


In [135]:
text_collection.find_one_and_update(
    {'_id': ObjectId('616fcef5e3381e0ff188026b')},
    {"$set":{'coord':{ 'type': "Point", 'coordinates': [ -20, -15 ] }}})


{'_id': ObjectId('616fcef5e3381e0ff188026b'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators',
 'text_name': 'Физики заперли свет в нанорезонаторе на рекордно долгое время',
 'art_text': '\nФизики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих пор добиться приемлемо

In [136]:
for a in text_collection.find({'_id': ObjectId('616fcef5e3381e0ff188026b')}):
    print(a)

{'_id': ObjectId('616fcef5e3381e0ff188026b'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': 'Физики заперли свет в нанорезонаторе на рекордно долгое время', 'art_text': '\nФизики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих пор добиться приемлемого 

In [137]:
for d in text_collection.find({'coord':{'$geoIntersects':{"$geometry": 
{"type": "Polygon",
 "coordinates": [[[15,10], [15,20],[25,20], [25,10], [15,10]]]
}}}}):
    print(d)

{'_id': ObjectId('616fcef4e3381e0ff188001c'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.Старт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсохода будет\xa0поиск соединени

#### Важное замечание

!!! Если не построить этот индекс, то искаться не будет!!!

In [138]:
#lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

text_collection.create_index([("coord", pymongo.GEOSPHERE)], unique=False)

'coord_2dsphere'

In [141]:
req=text_collection.find(
{
   "coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ 19.999 , 15.001 ]
       },
       "$maxDistance": 1000,
       "$minDistance": 0
     }
   }
}
)
#req
for d in req:    
    print(d)

{'_id': ObjectId('616fcef4e3381e0ff188001c'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.Старт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсохода будет\xa0поиск соединени

In [142]:
for d in text_collection.find({'coord':{'$geoWithin':{"$box": 
[[15,10], [25,20]]
}}}):
    print(d)

{'_id': ObjectId('616fcef4e3381e0ff188001c'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.Старт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсохода будет\xa0поиск соединени

In [143]:
#text_collection.create_index([('text_name', pymongo.ASCENDING)], unique=False)
text_collection.create_index([('art_text', pymongo.TEXT)], unique=False)


'art_text_text'

In [150]:
#for d in text_collection.find({'text_name': 'Бактерии помогли получить катализатор для электролиза воды'}):
# for d in text_collection.find({
#   "$text":{"$search": "воды"}}):
        
#for d in text_collection.find({'art_text':{'$regex':'воды'}}):
for d in text_collection.find({ "$text": { "$search": "Американские учеными" } } ):
# for d in text_collection.find({ "$text": { "$search": "Американ" } } ):
    print(d)

In [83]:
for d in text_collection.find():
    print(d)

{'_id': ObjectId('616fcef4e3381e0ff188001c'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.Старт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсохода будет\xa0поиск соединени